### Balancing Logistics Problems
We have to deliver products over the week the **6 days**, then we have to define 6 zones from given place from the **path_source**, however the solution is not that easy, we have some constraints. The stops and distribution of items needs to be balanced. Then we will combine **Constrained Kmeans** and **Integer Programming** (IP) or **Mixed Integer Linear Programming** (MILP).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pulp
#for haversine distance
from math import radians
#calculate distance matrix
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import haversine_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from k_means_constrained import KMeansConstrained

In [2]:
path_to_cplex = "/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux/cplex"

In [ ]:
path_milp = "milp_solution.csv"
path_ip = "ip_solution.csv"
path_source = "ubicaciones.csv"

In [3]:
df = pd.read_csv(path_source)

#### There are some values with no volume of delivery but they do have frequency, then we need to go through 
Those values that have no items to deliver will deliver 0.00001 items to be taken into account in the LP


In [4]:
df.loc[df[df["Vol_Entrega"] == 0].index, "Vol_Entrega"] = 0.00001

In [5]:
#based on the 6 working days of the week
zones = ["D1", "D2", "D3", "D4", "D5", "D6"]
agencies = list("A" + df["Id_Cliente"].astype(str))
vol_delivery = list(df["Vol_Entrega"])
vol_stores = list(df["Vol_Entrega"]*df["Frecuencia"])
frequency = list(df["Frecuencia"])
stores_volume = dict(zip(agencies, vol_stores))
stores_frequency = dict(zip(agencies, frequency))
vol_delivery = dict(zip(agencies, vol_delivery))

In [6]:
sum(vol_stores)/6

9066.66681333136

In [7]:
sum(frequency)/6
#df.shape[0]/6 # for balanced kmeans

662.8333333333334

In [8]:
def stops_gap(per):
    """
    Calculates the deviation in the stops. Numbers from cells above
    Args:
        per : Percentage of deviation from balanced stops
    Returns:
        upper : Upper bound
        lower : Lower bound
    """
    upper = int(663 + (663 * per))
    lower = int(662 - (662 * per))
    return(upper, lower)

def items_gap(per):
    """
    Calculates the deviation in the items distributed or flow. Numbers from cells above.
    Args:
        per : Percentage of deviation from distribution
    Returns:
        upper : Upper bound
        lower : Lower bound
    """
    upper = int(9067 + (9067 * per))
    lower = int(9066 - (9066 * per))
    return(upper, lower)

In [9]:
df["lat_radians"] = df["lat"].apply(lambda x: radians(x))
df["lon_radians"] = df["lon"].apply(lambda x: radians(x))

In [10]:
scaler = MinMaxScaler()
fitted_scaler = scaler.fit(df[["lat", "lon"]])
scaled_coordinates = fitted_scaler.transform(df[["lat", "lon"]])

### Weighted Kmeans vs. Balanced Kmeans
I have found using Balanced Kmeans is way better and results were good even without LP, IP or MILP optimization. It ss weight less complex than what we are doing below. I also think there might be other interesting ways of selecting centroids or trying to iterate and find minimum distances.<br>
We could implement a Kmeans were once the data constraints have been met, go to next cluster.

In [11]:
kmeans = KMeansConstrained(n_clusters=6,
                           size_min=600,
                           size_max=609,
                           random_state=12,
                           n_init=100,
                           max_iter=200,
                           n_jobs = -1)
kmeans_values = kmeans.fit(scaled_coordinates)
df["kmeans"] = list(kmeans.predict(scaled_coordinates))

In [10]:
#kmeans = KMeans(n_clusters = 6, random_state = 2020, init="random")
#kmeans_values = kmeans.fit_predict(scaled_coordinates, sample_weight=df["Vol_Entrega"])
#df["kmeans"] = list(kmeans_values)

In [12]:
vectorized_lat_lon = df[["lat", "lon"]].to_numpy()
#for haversine distance
#vectorized_lat_lon = df[["lat_radians", "lon_radians"]].to_numpy()

In [13]:
cluster_centers = fitted_scaler.inverse_transform(kmeans.cluster_centers_)

## Distance metrics (for more information give a look at README)
* Manhattan distance, also known as cityblock, largest distance.
* Euclidean, very common will not talk about it.
* Haversine distance, we are able to get km distance and works better for larger distances. However did not worked quiet properly on this specific case.

In [14]:
distance_matrix = cdist(cluster_centers, vectorized_lat_lon, metric= "euclidean")
#distance_matrix = haversine_distances(cluster_centers, vectorized_lat_lon) * (6371000/1000)

In [15]:
#create all possible routes
routes = [(z, a) for z in zones for a in agencies]

In [16]:
distances = pulp.makeDict([zones, agencies], distance_matrix, 0)

In [17]:
flow = pulp.LpVariable.dicts("Distribution", (zones, agencies), 0, None)

In [18]:
#same using either LpBinary or LpInteger
using = pulp.LpVariable.dicts("BelongstoZone", (zones, agencies), 0, 1, pulp.LpInteger)

In [18]:
prob = pulp.LpProblem("BrewingDatCup2020_v1", pulp.LpMinimize)

In [19]:
prob += pulp.lpSum([distances[z][a] * flow[z][a]  for (z, a) in routes]) + pulp.lpSum([distances[z][a] * using[z][a]  for (z, a) in routes]), "totalCosts"

In [22]:
for z in zones:
    prob += pulp.lpSum([using[z][a] for a in agencies]) <= 713, "SumStopsInZoneUpper %s"%z
    prob += pulp.lpSum([using[z][a] for a in agencies]) >= 612, "SumStopsInZoneLower %s"%z
    prob += pulp.lpSum([flow[z][a] for a in agencies]) <= 9748, "SumDistrInZoneUpper %s"%z
    prob += pulp.lpSum([flow[z][a] for a in agencies]) >= 8386, "SumDistrInZoneLower %s"%z

In [24]:
for z in zones:
    for a in agencies:
        prob += flow[z][a]-(100000*using[z][a]) <= 0

In [25]:
for z in zones:
    for a in agencies:
        prob += flow[z][a] <= vol_delivery[a]

In [26]:
#each equation its an agency
for a in agencies:
    prob += pulp.lpSum([flow[z][a] for z in zones]) >= stores_volume[a], "Distribution %s"%a

In [27]:
for a in agencies:
    prob += pulp.lpSum([using[z][a] for z in zones]) == stores_frequency[a], "FrequencyDistribution %s"%a

In [28]:
#if you got infeasability problems check your constraints
#https://coin-or.github.io/pulp/guides/how_to_configure_solvers.html
prob.writeLP("milp_brewing.lp")
solver = pulp.CPLEX_CMD(path=path_to_cplex)
prob.solve(solver)

1

In [29]:
print("Estado: ", pulp.LpStatus[prob.status])
print("Total Cost: ", pulp.value(prob.objective))

Estado:  Optimal
Total Cost:  1752.5996784102415


### Naive best solution
According to the given function to minimize

In [14]:
sum(distance_matrix.T.min(1) * vol_stores) + sum(distance_matrix.T.min(1) * frequency)

1563.6223089057698

## CPLEX Optimal Solutions
Estado:  Optimal (Extremely Balanced: 663 stops & 9067 items) <br>
Total Cost:  1783.0206990091006
<br>
<br>
Estado:  Optimal (10% balanced) <br>
Total Cost:  1748.4626671795415
<br>
<br>
Estado:  Optimal (7.5% balanced) <br>
Total Cost:  1752.5996784102415
<br>
<br>
Estado:  Optimal (5% balanced) <br>
Total Cost:  1758.3484761963682

#### Why using CPLEX 
Other solutions such as Pulp_CBC_CMD where ran for over 20 hours without being able to solve it. I have to add that there are easier ways to solve this.

In [30]:
final_df = pd.DataFrame(columns = ["D1", "D2", "D3", "D4", "D5", "D6"], index=(range(1, 3626)))

In [31]:
final_distr = dict()
for v in prob.variables():
    if (v.name).find("BelongstoZone_")==0:
        if v.varValue > 0:
            dist = v.name[14:]
            zone = dist[:2]
            id_cliente = int(dist[4:])
            final_df.loc[id_cliente, zone] = 1

In [32]:
final_df.fillna(0, inplace = True)
final_df = final_df.astype(int).reset_index().rename(columns = {"index":"Id_Cliente"})

In [33]:
final_df.to_csv(path_milp, header = True, index = False)

### Complete Integer Programming
Instead of adding a linking constraint, the way was donde above, multiply the binary value by the items to distribute and subject it to the max amount of items

In [19]:
prob = pulp.LpProblem("BrewingDatCup2020_v2", pulp.LpMinimize)

In [20]:
prob += pulp.lpSum([distances[z][a] * using[z][a]  for (z, a) in routes]), "totalCosts"

In [21]:
for a in agencies:
    prob += pulp.lpSum([using[z][a] for z in zones]) == stores_frequency[a], "FrequencyDistribution %s"%a

In [22]:
upper_stop, lower_stop = stops_gap(0.005)
upper_distr, lower_distr = items_gap(0.005)

In [23]:
for z in zones:
    prob += pulp.lpSum([using[z][a] for a in agencies]) <= upper_stop, "SumStopsInZoneUpper %s"%z
    prob += pulp.lpSum([using[z][a] for a in agencies]) >= lower_stop, "SumStopsInZoneLower %s"%z

In [24]:
for z in zones:
    prob += pulp.lpSum([using[z][a] * vol_delivery[a] for a in agencies]) <= upper_distr, "SumDistrInZoneUpper %s"%z
    prob += pulp.lpSum([using[z][a] * vol_delivery[a] for a in agencies]) >= lower_distr, "SumDistrInZoneLower %s"%z

In [25]:
#if you got infeasability problems check your constraints
#https://coin-or.github.io/pulp/guides/how_to_configure_solvers.html
prob.writeLP("ip_brewing.lp")
prob.solve()

1

In [26]:
print("Estado: ", pulp.LpStatus[prob.status])
print("Total Cost: ", pulp.value(prob.objective))

Estado:  Optimal
Total Cost:  90.23376746026902


In [27]:
final_df = pd.DataFrame(columns = ["D1", "D2", "D3", "D4", "D5", "D6"], index=(range(1, 3626)))

In [28]:
final_distr = dict()
for v in prob.variables():
    if (v.name).find("BelongstoZone_")==0:
        if v.varValue > 0:
            dist = v.name[14:]
            zone = dist[:2]
            id_cliente = int(dist[4:])
            final_df.loc[id_cliente, zone] = 1

In [29]:
final_df.fillna(0, inplace = True)
final_df = final_df.astype(int).reset_index().rename(columns = {"index":"Id_Cliente"})

In [30]:
final_df.to_csv(path_ip, header = True, index = False)

#### Check constrained_kmeans notebook to test results

In [30]:
#If problem in logic please test this
(np.argsort(distance_matrix.T))[1828]

array([4, 0, 1, 5, 3, 2])

## Further work
* Better way of measuring distances, for example TSP or mean centroid-agencies.
* Try different clustering constraints and seeds to see if improvements on score